In [2]:
import pandas as pd
import matplotlib as plt
import numpy as np
import os

In [3]:
df = pd.read_csv("../data/brasileirao_2009_limpo.csv")
df.columns

Index(['ID', 'rodada', 'data', 'hora', 'mandante', 'visitante', 'vencedor',
       'arena', 'mandante_Placar', 'visitante_Placar', 'mandante_Estado',
       'visitante_Estado', 'publico_jogo', 'juiz_partida', 'expulsao_mandate',
       'expulsao_visitante', 'dia_semana_jogo'],
      dtype='object')

In [4]:
# Parte 1 dos insights aqui faremos uma breve analise nos numeros do brasileirao, procurando a fundo sobre pontuacao, saldo de gols e resultados e como tudo isso influenciou para que os resultados acontecessem

def calcular_pontos_mandante(row):
    if row['mandante_Placar'] > row['visitante_Placar']:
        return 3
    if row['mandante_Placar'] == row['visitante_Placar']:
        return 1
    else:
        return 0
def calcular_pontos_visitante(row):
    if row['visitante_Placar'] > row['mandante_Placar']:
        return 3
    if row['visitante_Placar'] == row['mandante_Placar']:
        return 1
    else:
        return 0

df['pts_mandante'] = df.apply(calcular_pontos_mandante, axis=1)
df['pts_visitante'] = df.apply(calcular_pontos_visitante, axis=1)

def definir_resultado(row):
    if row['mandante_Placar'] > row['visitante_Placar']:
        return "Mandante"
    if row['mandante_Placar'] == row['visitante_Placar']:
        return "Empate"
    else:
        "Visitante"
df['status_vencedor'] = df.apply(definir_resultado, axis=1)
df

#TRES NOVAS COLUNAS PTS_MANDANTE PTS_VISITANTE e STATUS VENCENDOR!

,ID,rodada,data,hora,mandante,visitante,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado,publico_jogo,juiz_partida,expulsao_mandate,expulsao_visitante,dia_semana_jogo,pts_mandante,pts_visitante,status_vencedor
0,2707,1,2009-05-09,18:30,Sport,Barueri,Empate,Ilha do Retiro,1,1,PE,SP,16019,Francisco de Assis Almeida Filho,0,0,Sábado,1,1,Empate
1,2708,1,2009-05-09,18:30,Palmeiras,Coritiba,Palmeiras,Palestra Itália,2,1,SP,PR,19105,Arilson Bispo da Anunciação,0,0,Sábado,3,0,Mandante
2,2709,1,2009-05-09,18:30,Avai,Atletico-MG,Empate,Ressacada,2,2,SC,MG,8344,Rodrigo Nunes de Sá,0,0,Sábado,1,1,Empate
3,2710,1,2009-05-10,16:00,Corinthians,Internacional,Internacional,Pacaembu,0,1,SP,RS,14458,Evandro Rogério Roman,0,0,Domingo,0,3,None
4,2711,1,2009-05-10,16:00,Fluminense,Sao Paulo,Fluminense,Maracanã,1,0,RJ,SP,14574,Sandro Meira Ricci,0,0,Domingo,3,0,Mandante
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,3082,38,2009-12-06,17:00,Internacional,Santo Andre,Internacional,Beira Rio,4,1,RS,SP,0,Jailson Macedo Freitas,0,0,Domingo,3,0,Mandante
376,3083,38,2009-12-06,17:00,Santos,Cruzeiro,Cruzeiro,Vila Belmiro,1,2,SP,MG,6942,Marcelo de Lima Henrique,0,1,Domingo,0,3,None
377,3084,38,2009-12-06,17:00,Barueri,Athletico-PR,Empate,Prudentão,0,0,SP,PR,1741,Ricardo Marques Ribeiro,0,0,Domingo,1,1,Empate
378,3085,38,2009-12-06,17:00,Vitoria,Goias,Empate,Barradão,2,2,BA,GO,7974,Sálvio Spínola Fagundes Filho,0,0,Domingo,1,1,Empate


In [5]:
# Vamos Organizar agora as pontuacoes de times e baseado na tabela tentaremos encontrar ja o campeao de 2009 (Flamengo) e saber se a tabela esta correta.
tab_mandante = df.groupby("mandante")["pts_mandante"].sum().reset_index()
tab_mandante.columns = ['Time', 'PTS_Casa']

tab_visitante = df.groupby("visitante")["pts_visitante"].sum().reset_index()
tab_visitante.columns = ['Time', 'PTS_Fora']

tabela_2009 = pd.merge(tab_mandante, tab_visitante, on="Time")

tabela_2009["PTS_Total"] = tabela_2009["PTS_Casa"] + tabela_2009["PTS_Fora"] 
tabela_2009 = tabela_2009.sort_values(by="PTS_Total", ascending=False)

tabela_2009 = tabela_2009.reset_index()
tabela_2009["Posicao"] = tabela_2009.index+1


print("Classificacao Final de 2009:")
display(tabela_2009)


Classificacao Final de 2009:


,index,Time,PTS_Casa,PTS_Fora,PTS_Total,Posicao
0,8,Flamengo,40,27,67,1
1,12,Internacional,40,25,65,2
2,17,Sao Paulo,42,23,65,3
3,7,Cruzeiro,30,32,62,4
4,14,Palmeiras,40,22,62,5
5,2,Avai,36,21,57,6
6,1,Atletico-MG,32,24,56,7
7,10,Goias,33,22,55,8
8,11,Gremio,47,8,55,9
9,5,Corinthians,32,20,52,10


In [6]:
tabela_2009["Dependencia_Casa"] = (tabela_2009["PTS_Casa"] / tabela_2009["PTS_Total"]) * 100
tabela_2009 = tabela_2009.sort_values(by="Dependencia_Casa", ascending=False)
#Depedencia Casa esta em porcentagem para identificar qual foi a porcentagem de pontos que cada time fez em casa ou seja os times com maiores porcentagens so conseguiram fazer pontos dentro de casa.

print("Classificacao Final de 2009:")
display(tabela_2009)

Classificacao Final de 2009:


,index,Time,PTS_Casa,PTS_Fora,PTS_Total,Posicao,Dependencia_Casa
8,11,Gremio,47,8,55,9,85.454545
13,19,Vitoria,38,10,48,14,79.166667
19,18,Sport,24,7,31,20,77.419355
16,6,Coritiba,34,11,45,17,75.555556
11,3,Barueri,37,12,49,12,75.510204
18,13,Nautico,28,10,38,19,73.684211
17,15,Santo Andre,29,12,41,18,70.731707
12,0,Athletico-PR,33,15,48,13,68.750000
15,9,Fluminense,31,15,46,16,67.391304
14,4,Botafogo-RJ,31,16,47,15,65.957447


In [ ]:
#Notebook do total de Expulsos de cada time no campeonato brasileiro e expulsos dentro e fora de casa crucial para um dashboard que veremos mais a frente

expulsos_casa = df.groupby("mandante")["expulsao_mandate"].sum().reset_index()
expulsos_casa.columns = ['Time', 'Expulso_Casa']

expulsos_fora = df.groupby("visitante")["expulsao_visitante"].sum().reset_index()
expulsos_fora.columns = ['Time', 'Expulso_Fora']

total_expulsos = pd.merge(expulsos_casa, expulsos_fora, on="Time")

total_expulsos["Expulsos_Total"] = total_expulsos["Expulso_Casa"] + total_expulsos["Expulso_Fora"] 
total_expulsos = total_expulsos.sort_values(by="Expulsos_Total", ascending=False)

display(total_expulsos)

,Time,Expulso_Casa,Expulso_Fora,Expulsos_Total
18,Sport,7,9,16
7,Cruzeiro,8,7,15
17,Sao Paulo,3,11,14
10,Goias,5,8,13
13,Nautico,6,7,13
11,Gremio,2,10,12
19,Vitoria,1,10,11
12,Internacional,4,6,10
14,Palmeiras,3,7,10
4,Botafogo-RJ,7,2,9


In [20]:
#transformando a hora em INTEIRO para conseguirmos trabalhar melhor nas somas de horarios dos jogos, dias

df["hora_limpa"] = df["hora"].str.split(":").str[0].astype(int)

df["total_gols"] = df["mandante_Placar"] + df["visitante_Placar"]

print(df.groupby("hora_limpa")["total_gols"].mean())

hora_limpa
16    2.869159
17    3.800000
18    2.758389
19    2.250000
21    3.070423
Name: total_gols, dtype: float64


In [ ]:
jogos_por_horario = df["hora_limpa"].value_counts()
print(jogos_por_horario)
#jogos em cada horario do brasileirao

hora_limpa
18    149
16    107
21     71
19     28
17     25
Name: count, dtype: int64


In [ ]:
df_turno1 = df[df["rodada"] <= 19]

casa_t1 = df_turno1.groupby("mandante")["pts_mandante"].sum().reset_index()
casa_t1.columns = ["Time", "PTS_Casa_T1"]

fora_t1 = df_turno1.groupby("visitante")["pts_visitante"].sum().reset_index()
fora_t1.columns = ["Time", "PTS_Fora_T1"]

tabela_t1 = pd.merge(casa_t1, fora_t1, on="Time")
tabela_t1["Total_T1"] = tabela_t1["PTS_Casa_T1"] + tabela_t1["PTS_Fora_T1"]

df_turno2 = df[df["rodada"] > 19]

casa_t2 = df_turno2.groupby("mandante")["pts_mandante"].sum().reset_index()
casa_t2.columns = ["Time", "PTS_Casa_T2"]

fora_t2 = df_turno2.groupby("visitante")["pts_visitante"].sum().reset_index()
fora_t2.columns = ["Time", "PTS_Fora_T2"]

tabela_t2 = pd.merge(casa_t2, fora_t2, on="Time")
tabela_t2["Total_T2"] = tabela_t2["PTS_Casa_T2"] + tabela_t2["PTS_Fora_T2"]

comparativo_turnos = pd.merge(tabela_t1[["Time", "Total_T1"]], tabela_t2[["Time", "Total_T2"]], on="Time")
comparativo_turnos['Evolucao'] = comparativo_turnos['Total_T2'] - comparativo_turnos['Total_T1']
display(comparativo_turnos)

# Comparacao de quanto os times EVOLUIRAM do turno 1 PARA o turno 2

,Time,Total_T1,Total_T2,Evolucao
0,Athletico-PR,24,24,0
1,Atletico-MG,32,24,-8
2,Avai,30,27,-3
3,Barueri,28,21,-7
4,Botafogo-RJ,21,26,5
5,Corinthians,28,24,-4
6,Coritiba,19,26,7
7,Cruzeiro,22,40,18
8,Flamengo,27,40,13
9,Fluminense,15,31,16


In [ ]:
tabela_completa = pd.merge(tabela_2009, total_expulsos[['Time', 'Expulsos_Total']], on="Time")

tabela_completa = pd.merge(tabela_completa, comparativo_turnos[['Time', 'Evolucao']], on="Time")

tabela_completa['Dependencia_Casa'] = (tabela_completa['PTS_Casa'] / tabela_completa['PTS_Total']) * 100

display(tabela_completa)

tabela_completa.to_csv("ranking_final_consolidado_2009.csv", index=False)
df.to_csv("brasileirao_2009_jogos_final.csv", index=False)

#Salvando todas as tabelas e o CSV atualizado para os proximos Notebooks

,index,Time,PTS_Casa,PTS_Fora,PTS_Total,Posicao,Dependencia_Casa,Expulsos_Total,Evolucao
0,11,Gremio,47,8,55,9,85.454545,12,-1
1,19,Vitoria,38,10,48,14,79.166667,11,-2
2,18,Sport,24,7,31,20,77.419355,16,5
3,6,Coritiba,34,11,45,17,75.555556,6,7
4,3,Barueri,37,12,49,12,75.510204,8,-7
5,13,Nautico,28,10,38,19,73.684211,13,2
6,15,Santo Andre,29,12,41,18,70.731707,9,5
7,0,Athletico-PR,33,15,48,13,68.750000,6,0
8,9,Fluminense,31,15,46,16,67.391304,8,16
9,4,Botafogo-RJ,31,16,47,15,65.957447,9,5
